[참고사이트](https://yurimkoo.github.io/python/2019/09/14/connect-db-with-python.html)

In [33]:
import pymysql
import pandas as pd

# DB 연결

In [2]:
local_db = pymysql.connect(
    user='root',
    passwd='DKrmemf!@34',
    host="127.0.0.1",
    db="test_db",
    charset='utf8'
)

---

# cursor 설정
1. `cursor()` : `연결한 DB와 상호작용`하는 `cursor객체`를 생성한다.
2. DB의 데이터를 Dict형태로 가져오는 `DictCursor`를 사용
    + Python에서 데이터 분석시 `pandas`를 이용  
    + 관계형 데이터베이스 시스템(`RDBMS`)를 주로 사용
3. cursor 객체의 SQL문 실행문(`cursor.execute(sql)`) 이후 메서드
    + fetchall()  : 모든 데이터를 한번에 가져올때 사용
    + fetchone()  : 한 번 호출에 하나의 행만 가져올 때 사용
    + fetchmany(n): n개만큼의 데이터를 가져올 때 사용

In [3]:
cursor = local_db.cursor(pymysql.cursors.DictCursor) # local_db와 상호작용하는 cursor객체를 생성

In [5]:
sql = "SELECT * FROM test_table"
cursor.execute(sql)
result = cursor.fetchall()

print(type(result))
print(type(result[0]))
print(result)
print(pd.DataFrame(result))

<class 'list'>
<class 'dict'>
[{'id': 45, 'name': 'LEE'}, {'id': 46, 'name': 'LEE'}, {'id': 47, 'name': 'Park'}, {'id': 48, 'name': 'Park'}, {'id': 49, 'name': 'Park'}, {'id': 51, 'name': 'JONG WITH ID > 50'}, {'id': 52, 'name': 'JONG WITH ID > 50'}, {'id': 53, 'name': 'JONG WITH ID > 50'}]
   id               name
0  45                LEE
1  46                LEE
2  47               Park
3  48               Park
4  49               Park
5  51  JONG WITH ID > 50
6  52  JONG WITH ID > 50
7  53  JONG WITH ID > 50


In [16]:
result2df = pd.DataFrame(result)
print(result2df)
result2df.head()

   id  name
0  45   LEE
1  46   LEE
2  47  Park
3  48  Park
4  49  Park


,id,name
0,45,LEE
1,46,LEE
2,47,Park
3,48,Park
4,49,Park


---

# 데이터 처리

## 삽입(INSERT), 변경(UPDATE), 삭제(DELETE)

### INSERT

In [22]:
sql = \
"""
INSERT INTO test_table (id, name)
VALUE (0, "LEE");
"""

cursor.execute(sql) # sql문을 실행한다.
local_db.commit()   # 결과가 DB에 반영된다.

In [35]:
cursor.execute("SELECT * FROM test_table;")
# print(cursor.fetchall())
print(cursor.fetchone())

{'id': 45, 'name': 'LEE'}


In [48]:
print(pd.DataFrame(cursor.fetchone(), index=[0]))

Empty DataFrame
Columns: []
Index: [0]


### UPDATE

In [6]:
sql = \
"""
SELECT * FROM test_table
"""

cursor.execute(sql)
print(pd.DataFrame(cursor.fetchall()))

   id               name
0  45                LEE
1  46                LEE
2  47               Park
3  48               Park
4  49               Park
5  51  JONG WITH ID > 50
6  52  JONG WITH ID > 50
7  53  JONG WITH ID > 50


In [7]:
sql = \
"""
UPDATE test_table
SET name='Kim WITH ID < 50'
WHERE id < 50;
"""

cursor.execute(sql)
# local_db.commit()

5

In [8]:
sql = \
"""
SELECT * FROM test_table
"""

cursor.execute(sql)
print(pd.DataFrame(cursor.fetchall()))

   id               name
0  45   Kim WITH ID < 50
1  46   Kim WITH ID < 50
2  47   Kim WITH ID < 50
3  48   Kim WITH ID < 50
4  49   Kim WITH ID < 50
5  51  JONG WITH ID > 50
6  52  JONG WITH ID > 50
7  53  JONG WITH ID > 50


In [9]:
local_db.commit()

### DELETE

In [10]:
sql = \
"""
SELECT * FROM test_table
"""
cursor.execute(sql)
print(pd.DataFrame(cursor.fetchall()))

   id               name
0  45   Kim WITH ID < 50
1  46   Kim WITH ID < 50
2  47   Kim WITH ID < 50
3  48   Kim WITH ID < 50
4  49   Kim WITH ID < 50
5  51  JONG WITH ID > 50
6  52  JONG WITH ID > 50
7  53  JONG WITH ID > 50


In [11]:
sql = \
"""
DELETE FROM test_table
WHERE id BETWEEN 47 AND 51
"""

cursor.execute(sql)

4

In [15]:
sql = \
"""
SELECT * FROM test_table
"""
cursor.execute(sql)
print(pd.DataFrame(cursor.fetchall()))

   id               name
0  45   Kim WITH ID < 50
1  46   Kim WITH ID < 50
2  52  JONG WITH ID > 50
3  53  JONG WITH ID > 50


In [14]:
local_db.commit()

+ commit `전에 실행했던 SQL문을 취소`하려면 어떻게?

---

# Placeholder 사용하기

## TIP. execute() / executemany() 에 placeholder 사용?
1. Q. 만약 DB내의 데이터에 대해 `대량` 삽입/변경/삭제가 필요한데, `조건이 모두 다르다면`?
2. A. `Placeholder`를 사용한다!
3. 이용방식
    + execute(sql, a_data)
    + executemany(sql, multiple_data)
    + 단, sql문은 `%s`등을 이용하여 동적값이 들어가게 만듬. (일반적인 문자열에 들어가는 %s와는 다르다.)
4. 장점
    + `반복문 + execute()`보다 속도, 메모리면에서 훨씬 뛰어나다.

### execute()

In [17]:
a_data = (45, "KIM WITH ID < 50")

# SELECT
sql = """SELECT * FROM test_table WHERE id=%s AND name=%s;"""
cursor.execute(sql, a_data)
print(pd.DataFrame(cursor.fetchall(), index=[0]), end="\n\n\n")

# DELETE
sql = """DELETE FROM test_table WHERE id=%s AND name=%s"""
cursor.execute(sql, a_data)
sql = """SELECT * FROM test_table;"""
cursor.execute(sql)
print(pd.DataFrame(cursor.fetchall()))

# local_db.commit()

   id              name
0  45  Kim WITH ID < 50


   id               name
0  46   Kim WITH ID < 50
1  52  JONG WITH ID > 50
2  53  JONG WITH ID > 50


### executemany()

In [29]:
sql = """SELECT * FROM test_table;"""
cursor.execute(sql)
print(pd.DataFrame(cursor.fetchall()))

   id  name
0   1  SUNG
1   2   GUI
2   3  JANG
3  46  SUNG
4  52   GUI
5  53  JANG


In [31]:
datas = [[1, "SUNG"], [2, "GUI"], [3, "JANG"]]

## DELETE
sql = "DELETE FROM test_table WHERE id=%s AND name=%s"
cursor.executemany(sql, datas)
sql = """SELECT * FROM test_table;"""
cursor.execute(sql)
print(pd.DataFrame(cursor.fetchall()), end="\n\n\n")

## INSERT
sql = "INSERT INTO test_table(id, name) VALUES (%s, %s)"
cursor.executemany(sql, datas)
sql = """SELECT * FROM test_table;"""
cursor.execute(sql)
print(pd.DataFrame(cursor.fetchall()), end="\n\n\n")
# local_db.commit()

datas = [[46, "SUN"], [52, "GUI"], [53, "JANG"]]
datas = [data[::-1] for data in datas]
## UPDATE
sql = "UPDATE test_table SET name=%s WHERE id=%s"
cursor.executemany(sql, datas)
sql = "SELECT * FROM test_table"
cursor.execute(sql)
print(pd.DataFrame(cursor.fetchall()))

   id  name
0  46  SUNG
1  52   GUI
2  53  JANG


   id  name
0   1  SUNG
1   2   GUI
2   3  JANG
3  46  SUNG
4  52   GUI
5  53  JANG


   id  name
0   1  SUNG
1   2   GUI
2   3  JANG
3  46   SUN
4  52   GUI
5  53  JANG


In [32]:
sql = "SELECT * FROM test_table"
cursor.execute(sql)
print(pd.DataFrame(cursor.fetchall()))

   id  name
0   1  SUNG
1   2   GUI
2   3  JANG
3  46   SUN
4  52   GUI
5  53  JANG


In [34]:
local_db.close()

---

In [ ]:
connect_to_localhost = pymysql.connect(host='localhost', 
                                       user='root',
                                       password='DKrmemf!@34',
                                       db='test_db',
                                       charset='utf8')

In [ ]:
try :
    cursor = connect_to_localhost.cursor(pymysql.cursors.DictCursor)
    sql = "SELECT * FROM test_table"
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    for row in rows :
        print(row)
        
except Exception as e:
    print(str(e))
    
finally :
    connect_to_localhost.close()

##
# {'id': 45, 'name': 'Kim WITH ID < 50'}
# {'id': 46, 'name': 'Kim WITH ID < 50'}
# {'id': 52, 'name': 'JONG WITH ID > 50'}
# {'id': 53, 'name': 'JONG WITH ID > 50'}

In [ ]:
try :
    cursor = connect_to_localhost.cursor(pymysql.cursors.DictCursor)
    sql = "INSERT INTO test_table(id, name) VALUES (%s, %s)"
    
    cursor.execute(sql, (0, "PEE"))
    cursor.execute(sql, (0, "Yoo"))
    cursor.execute(sql, (0, "Park"))
    
    sql = "SELECT * FROM test_table"
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    for row in rows :
        print(row)
        
except Exception as e:
    print(str(e))
    
finally :
    connect_to_localhost.close()
    
##
# {'id': 45, 'name': 'Kim WITH ID < 50'}
# {'id': 46, 'name': 'Kim WITH ID < 50'}
# {'id': 52, 'name': 'JONG WITH ID > 50'}
# {'id': 53, 'name': 'JONG WITH ID > 50'}
# {'id': 54, 'name': 'PEE'}
# {'id': 55, 'name': 'Yoo'}
# {'id': 56, 'name': 'Park'}

In [ ]:
try :
    cursor = connect_to_localhost.cursor(pymysql.cursors.DictCursor)
    
    sql = "SELECT * FROM test_table"
    cursor.execute(sql)
    print(pd.DataFrame(cursor.fetchall()), end="\n\n\n")
    
    sql = "UPDATE test_table SET name='Kim' WHERE id < 50"
    cursor.execute(sql)
    
    sql = "SELECT * FROM test_table"
    cursor.execute(sql)
    print(pd.DataFrame(cursor.fetchall()))
    
except Exception as e :
    print(str(e))
finally :
    connect_to_localhost.close()
    

##
#    id               name
# 0  45   Kim WITH ID < 50
# 1  46   Kim WITH ID < 50
# 2  52  JONG WITH ID > 50
# 3  53  JONG WITH ID > 50


#    id               name
# 0  45                Kim
# 1  46                Kim
# 2  52  JONG WITH ID > 50
# 3  53  JONG WITH ID > 50

In [ ]:
try :
    cursor = connect_to_localhost.cursor(pymysql.cursors.DictCursor)
    
    sql = "SELECT * FROM test_table"
    cursor.execute(sql)
    print(pd.DataFrame(cursor.fetchall()), end="\n\n\n")
    
    sql = "DELETE FROM test_table WHERE name=%s"
    cursor.execute(sql, "Kim WITH ID < 50")
    
    sql = "SELECT * FROM test_table"
    cursor.execute(sql)
    print(pd.DataFrame(cursor.fetchall()))
    
except Exception as e :
    print(str(e))
    
finally :
    connect_to_localhost.close()
    
##
#    id               name
# 0  45   Kim WITH ID < 50
# 1  46   Kim WITH ID < 50
# 2  52  JONG WITH ID > 50
# 3  53  JONG WITH ID > 50


#    id               name
# 0  52  JONG WITH ID > 50
# 1  53  JONG WITH ID > 50